In [5]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN, CSVLogger
from keras import backend as K

from ssd300 import build_SSD300
import Decoder
import Encoder

In [10]:
# Configuration of the model: 
input_shape=(300, 300, 3)
numClasses = 10
iou_thres=0.5 # for default and gt matching
nms_thres=0.45 # IoU threshold for non-maximal suppression
score_thres=0.01 # threshold for classification scores
top_k=200 # the maximum number of predictions kept per image
min_scale=0.2 # the smallest scale of the feature map
max_scale=0.9 # the largest scale of the feature map
aspect_ratios=[0.5, 1, 2] # aspect ratios of the default boxes to be generated
n_predictions=6 # the number of prediction blocks
prediction_size=[37, 18, 10, 5, 3, 1] # sizes of feature maps at each level

### Parse Input: 


In [6]:
# 
# x_train, y_train = parse_json()

### Ecode Parsed Input: 
In this step, we perform bipartite matching and multi-matching algorithm to match default boxes generated with the ground-truth labels of our training data. 

In [15]:
ssd = build_SSD300(input_shape=input_shape, 
                  numClasses=numClasses, 
                  mode='training', 
                  min_scale=min_scale, 
                  max_scale=max_scale, 
                  aspect_ratios=aspect_ratios, 
                  iou_thres=iou_thres,
                  nms_thres=nms_thres, 
                  score_thres=score_thres, 
                  top_k=top_k,
                  n_predictions=n_predictions)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam, loss=loss_function, 
             metrics=['accuracy'])


TypeError: build_SSD300() got an unexpected keyword argument 'nms_thres'

In [9]:
# Initialize an Ecoder object
encode = Encoder(y_truth=y_train, 
                numClasses=numClasses, 
                iou_thres=iou_thres,
                min_scale=min_scale, 
                max_scale=max_scale, 
                aspect_ratio=aspect_ratios, 
                n_predictions=n_predictions, 
                prediction_size=prediction_size)

# Encode the labels and ground-truth boxes of the training images
y_train = encode.get_encoded_data()

NameError: name 'y_train' is not defined